In [86]:
from __future__ import print_function 
from bs4 import BeautifulSoup
from BeautifulSoup import SoupStrainer as sopstrain
import subprocess
import os
import re
import sys
import numpy as np
import string
import operator
import csv
import urllib2
import json
from collections import defaultdict
import StringIO
import random
from multiprocessing import Pool

In [87]:
knowledgGraph = "cnet" # conceptnet5AssocMod, gov, conceptnet5AssocMi, conceptnet5AssocHdl
#knowledgGraph_ = ''.join([k.capitalize() if i>0 else k for i, k in enumerate(knowledgGraph)])
collection = "trec7n8"
method = "wsdmImpr" # hal, mi, assoc, assocMi, assocHal, assoc2
simMeasure = "" # mi, cnet
projectDir = "/home/fj9124/projects/ir/seq_kb_ir/" 
indexDir = "/scratch/index/indri_5_7/"
colMethodConfigsDir = os.path.join(projectDir, "configs", collection, method)
cfgInFileName = os.path.join(projectDir, "configs", collection, "query" + collection.capitalize()) 
print("cfgInFileName =", cfgInFileName)
cfgOutFileName=os.path.join(colMethodConfigsDir, knowledgGraph, "indriRunQuery.cfg") 
print("cfgOutFileName =", cfgOutFileName)
colIndexDir = os.path.join(indexDir, collection) 
print("colIndexDir =", colIndexDir)
if knowledgGraph in {"cnet"}:
    knowledgGraphIndexDir = os.path.join(indexDir, collection)   
else:
    knowledgGraphIndexDir = os.path.join(indexDir, knowledgGraph)   
print("knowledgGraph index Dir =", knowledgGraphIndexDir)
graphsDir = os.path.join(projectDir, "graphs")
#methodGraphsDir = os.path.join(graphsDir, method)
methodGraphsFileName = []
if method == "hal":
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph + ".txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method.translate(None, string.digits) in {"mi", "assoc"}:
    methodGraphsFileName = [os.path.join(graphsDir, method.translate(None, string.digits), knowledgGraph, "graph.txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocMi"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "mi", collection, "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocHal"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "hal", "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
qrelsDir = os.path.join(projectDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
colQrelsFileName = os.path.join(colQrelsDir, "qrels.csv")
print("col Qrels File Name =", colQrelsFileName)
runsFileName = os.path.join(projectDir, "runs", method, collection, knowledgGraph, "indriRunQuery.runs") 
print("runsFileName =", runsFileName)
evalsFileName = os.path.join(projectDir, "evals", method, collection, knowledgGraph, "indriRunQuery.evals") 
print("evalsFileName =", evalsFileName)
dumpindexStatementFilename = os.path.join("/", "tmp", "sbsb", "statement" + "_" + collection + "_" + method + "_" + knowledgGraph )
print("dumpindexStatementFilename =", dumpindexStatementFilename)
occuranceCountFilename = os.path.join(projectDir, "occuranceCount", "occuranceCount")
print("occuranceCountFilename =", occuranceCountFilename)
countsResultsFile = os.path.join(projectDir,"occuranceCount","results",collection+".txt")
print("countsResultsFile =", countsResultsFile)
conceptnet5RelAllFilename ="/scratch/saeid/data/conceptnet5_simp.csv"
print("conceptnet5RelAllFilename =", conceptnet5RelAllFilename)
cfgTmpOutFileName = cfgOutFileName + ".tmp"
print("cfgTmpOutFileName =", cfgTmpOutFileName)
docIdNameMapFileName = os.path.join(projectDir, "occuranceCount", "results", collection + "_docIdNameMap.txt")
print("docIdNameMapFileName =", docIdNameMapFileName)
#dumpindexResFilename = os.path.join(projectDir,"occuranceCount","results",collection +"_dumpindexRes"+".txt")
#print("dumpindexResFilename =", dumpindexResFilename)
#expansionCount = 25

cfgInFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/queryTrec7n8
cfgOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/wsdmImpr/cnet/indriRunQuery.cfg
colIndexDir = /scratch/index/indri_5_7/trec7n8
knowledgGraph index Dir = /scratch/index/indri_5_7/trec7n8
col Qrels File Name = /home/fj9124/projects/ir/seq_kb_ir/qrels/trec7n8/qrels.csv
runsFileName = /home/fj9124/projects/ir/seq_kb_ir/runs/wsdmImpr/trec7n8/cnet/indriRunQuery.runs
evalsFileName = /home/fj9124/projects/ir/seq_kb_ir/evals/wsdmImpr/trec7n8/cnet/indriRunQuery.evals
dumpindexStatementFilename = /tmp/sbsb/statement_trec7n8_wsdmImpr_cnet
occuranceCountFilename = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/occuranceCount
countsResultsFile = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/trec7n8.txt
conceptnet5RelAllFilename = /scratch/saeid/data/conceptnet5_simp.csv
cfgTmpOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/wsdmImpr/cnet/indriRunQuery.cfg.tmp
docIdNameMa

In [88]:
indriRunQuery_hist_= dict()
dumpindex_hist_ = dict()

In [89]:
# trec_eval -q ../qrels/trec7n8/qrels.csv indriRunQuery.res | grep map | grep -v all | awk '{if ($3 < 0.1) printf $2 ","}'
if collection == "gov":
    diffTopics = [4, 6, 7, 8, 9, 10, 12, 18, 19, 21, 22, 24, 26, 27, 30, 31, 32, 33, 36, 37, 40, 45, 46, 47, 48, 50, 52, 53, 54, 57, 58, 59, 63, 65, 66, 67, 70, 73, 78, 79, 80, 82, 84, 86, 92, 93, 94, 95, 96, 98, 99, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 115, 116, 117, 118, 120, 123, 125, 126, 127, 129, 132, 133, 134, 135, 136, 137, 140, 142, 143, 147, 148, 149, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 165, 166, 167, 168, 169, 171, 175, 176, 177, 178, 179, 180, 182, 183, 185, 186, 188, 189, 191, 192, 194, 195, 196, 197, 200, 201, 202, 203, 204, 207, 208, 211, 212, 214, 215, 218, 219, 221, 224]
if collection == "robust":
    diffTopics = [301, 305, 309, 314, 315, 318, 319, 320, 322, 325, 327, 332, 336, 340, 342, 343, 344, 345, 346, 347, 350, 352, 354, 355, 356, 359, 363, 367, 371, 372, 376, 378, 379, 380, 381, 383, 386, 388, 389, 390, 393, 394, 397, 398, 401, 405, 409, 412, 419, 421, 426, 432, 435, 436, 437, 439, 440, 442, 448, 449, 605, 608, 610, 620, 622, 625, 626, 627, 638, 650, 651, 655, 659, 666, 668, 680, 683, 684, 688, 689, 690]
if collection == "trec7n8":
    diffTopics = [352,354,356,359,363,367,370,371,372,373,376,378,379,381,383,386,388,389,390,393,394,397,398,401,405,408,419,421,422,426,432,435,436,437,439,440,442,443,445,447,448,449]
diffTopic = [str(i) for i in diffTopics]

In [90]:
# generate docIdNameMapFileName
if not os.path.isfile(occuranceCountFilename):
    cmd = ' '.join([occuranceCountFilename, colIndexDir, "dm", ">", docIdNameMapFileName])
    res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()

In [91]:
with open(docIdNameMapFileName, 'r') as f:
    reader = list(csv.reader(f, delimiter = ' '))
    print(reader[0])
    docIdNameMap = {i[1]:i[0] for i in reader}
    docNameIdMap = {i[0]:i[1] for i in reader}
for i, (k, v) in enumerate(docIdNameMap.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

['1', 'FR940104-1-00001']
LA082389-0020 	 441223
FBIS3-32596 	 246524
LA011890-0198 	 465144
LA011890-0199 	 465145
LA011890-0196 	 465142
LA011890-0197 	 465143
LA011890-0194 	 465140
FT932-3799 	 88713
LA011890-0192 	 465138
LA011890-0193 	 465139
LA011890-0190 	 465136
LA011890-0191 	 465137
FT932-3796 	 88710
FT924-2868 	 187695
FT924-2869 	 187696
FT932-3797 	 88711
FT924-2864 	 187691
FT924-2865 	 187692
FT924-2866 	 187693
FT924-2867 	 187694
FT924-2860 	 187687
FT924-2861 	 187688


In [92]:
count_history = dict()
if os.path.isfile(countsResultsFile):
    with open(countsResultsFile, 'r') as f:
        reader = csv.reader(f, delimiter = "\t")
        count_history = {k:int(float(v)) for k,v in list(reader)}
    print("size of count_history =", len(count_history))
    for i, (k, v) in enumerate(count_history.iteritems()):
        print (k, '\t', v)
        if (i>20):
            break

size of count_history = 81446
#uw(#4( storm ) #4( be appear in sky )) 	 0
#uw(#4( work on sundai )) 	 137
#uw(#4( calm time )) 	 23
#uw(#4( kill ) #4( prai )) 	 394
#uw(#4( bank ) #4( teller offic )) 	 5
#uw(#4( water ) #4( hydrogen )) 	 347
#uw(#4( drug ) #4( take )) 	 8847
#uw(#4( space ) #4( mostli empti )) 	 3
#uw(#4( chang ) #4( edg )) 	 3803
#uw(#4( happin )) 	 0
#uw(#4( trade ) #4( countri )) 	 45838
#uw(#4( game ) #4( leisur activ )) 	 23
#uw(#4( round in shape )) 	 4
#uw(#4( good ) #4( other person )) 	 411
#uw(#4( smoke ) #4( fire )) 	 1842
#uw(#4( spice up salad )) 	 1
#uw(#4( be wound )) 	 410
#uw(#4( good ) #4( hear nice new )) 	 0
#uw(#4( teacher ) #4( teach student of class )) 	 0
#uw(#4( deliveri )) 	 11268
#uw(#4( bank ) #4( teller monei )) 	 23
#uw(#4( swap share )) 	 106


In [93]:
conceptnet5RelAll = defaultdict(dict)
conceptnet5RelAllInv = defaultdict(dict)
with open(conceptnet5RelAllFilename, 'r') as f:
    reader = csv.reader(f, delimiter = ",")
    for line in list(reader):
        if line[0].strip() != "" and line[1].strip() != "" and line[2].strip() != "":
            if all(c in string.printable for c in line[0]) and all(c in string.printable for c in line[1]) and all(c in string.printable for c in line[2]):
                conceptnet5RelAll[line[1].strip()][line[2].strip()] = line[0].strip()
                conceptnet5RelAllInv[line[2].strip()][line[1].strip()] = line[0].strip()
print("size of conceptnet5RelAll =", len(conceptnet5RelAll))
for i, (k1, k2v) in enumerate(conceptnet5RelAll.iteritems()):
    for j, (k2, v) in enumerate(k2v.iteritems()):
        if (i>3 or j>3):
            break
        print (k1, '\t', k2, '\t', v)
        

size of conceptnet5RelAll = 2965684
tripolitan 	 tripoline 	 Synonym
tripolitan 	 tripoli 	 RelatedTo
age of nemesis 	 band 	 IsA
age of nemesis 	 organisation 	 InstanceOf
age of nemesis 	 progressive rock 	 dbpedia/genre
age of nemesis 	 progressive metal 	 dbpedia/genre
joseph john annabring 	 person 	 InstanceOf
british rail class 438 	 mean of transportation 	 InstanceOf
british rail class 438 	 train 	 InstanceOf


In [124]:
def splitStemText(text):
    stemmedWords = []
    text = re.sub('/|-|\"|_',' ', text) # replace - and slash with space
    if method in {"assocRestful", "lr", "wsdmImpr"}:
        for text_ in text.split(' '):
            text = text_.replace(" ", "_")
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            text = os.path.basename(data_j[u'uri'])
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            #print("data_j =", data_j)
            #print("data_j[u'uri'] =", data_j[u'uri'])
            stemmedWords += [str(os.path.basename(data_j[u'uri']))]
        #print("stemmedWords =", stemmedWords)
        return (stemmedWords)
    else:
        words = text.split()
        for w in words:
            w = re.sub('\(|\)|\'s|,','', w) # remove paranthesis, apostrophe s, comma
            w = re.sub('\'','', w) # remove apostrophe
            cmd = "dumpindex " + knowledgGraphIndexDir + " t " + w + " | head -n1"
            #print("cmd =", cmd)
            stemmedWords.append(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().split()[1])
        return (stemmedWords)

print ("testing:", splitStemText("going reading"))

testing: ['go', 'read']


In [125]:
def analyseQueries():
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soup = BeautifulSoup(reader, 'lxml')
        if collection in {"aquaint", "gov"}:
            tags = soup.find_all(['doc'])
        elif collection in {"robust", "trec7n8", "gov2"}:
            tags = soup.find_all(['top'])
            #print(tags)
        
        origWordsAll = dict()
        for tag in tags:
            
            if collection in {"aquaint", "gov"}:
                docno = (tag.find('docno').string).strip()
            elif collection in {"robust", "trec7n8", "gov2"}:
                result = re.search('<num> Number: (.*)\n', str(tag))                
                docno = result.group(1).strip()
            
            if collection in {"aquaint", "gov"}:
                origWords = splitStemText(tag.find('text').string.strip())
            elif collection in {"robust", "trec7n8", "gov2"}:
                result = re.search('<title>(.*)\n', str(tag))                
                origWords = splitStemText(result.group(1).strip())
            #print("origWords =", origWords)
            origWordsAll[docno] = origWords
            
    return (origWordsAll)

In [126]:
cmd = ' '.join(["dumpindex", colIndexDir, "s", "|", "awk", "\'NR==2{print $2}\'"])
res = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
colDocumentCount = int(np.float(res))
print("colDocumentCount =", colDocumentCount)

colDocumentCount = 472526


In [127]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genMultSimpQuery(argss):
    #    #print("intCoeff =", intCoeff)
    soupNew = BeautifulSoup("", 'lxml')
    parameters_tag = soupNew.new_tag("parameters")
    index_tag = soupNew.new_tag("index")
    index_tag.string = colIndexDir
    parameters_tag.append(index_tag)
        #print(tags)

    for args in argss:
        #print("args =", args)
        (docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights) = args
        #print("(...) =", docno, origWords_, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        doc_tag = soupNew.new_tag("query")

        docno_tag = soupNew.new_tag("number")
        docno_tag.string = '_'.join([docno, ('_'.join(origWords_)).replace(' ', '_'), relW.replace(' ', '_')])
        doc_tag.append(docno_tag)

        text_tag = soupNew.new_tag("text")
        text_tag.string = "#weight(\n" 

        text_tag.string += str(intCoeff0) + " #combine(" 
        for ow in set(origWords_):
            ow = regex.sub(' ', ow)
            text_tag.string += ow + " "
        text_tag.string += ")\n"

        if len(relW)>0: 
            relText_string1 = regex.sub(' ', relW)
            text_tag.string += str(intCoeff1) + " #combine(" 
            #print("relText_string1 =", relText_string1)
            text_tag.string += relText_string1.encode('utf-8')
            text_tag.string += ")\n"

        text_tag.string += ") "

        doc_tag.append(text_tag)

        parameters_tag.append(doc_tag)
        #print(doc_tag)

    rule_tag = soupNew.new_tag("rule")
    rule_tag.string = "method:dir,mu:" + str(dirCoeff)
    #rule_tag.string = "method:two"
    parameters_tag.append(rule_tag)

    #intCoeff_tag = soupNew.new_tag("intCoeff")
    #intCoeff_tag.string = "0.8"
    #parameters_tag.append(intCoeff_tag)

    threads_tag = soupNew.new_tag("threads")
    threads_tag.string = "32"
    parameters_tag.append(threads_tag)

    count_tag = soupNew.new_tag("count")
    count_tag.string = "10"
    parameters_tag.append(count_tag)

    trecFormat_tag = soupNew.new_tag("trecFormat")
    trecFormat_tag.string = "true"
    parameters_tag.append(trecFormat_tag)

    soupNew.append(parameters_tag)
    #print(soupNew.prettify())
    #print("cfgTmpOutFileName =", cfgTmpOutFileName)
    with open( cfgTmpOutFileName, 'a+') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)

In [128]:
def weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):
    
    indriRunQuery_hist = [(docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, cfgTmpOutFileName)]
    return indriRunQuery_hist

In [129]:
def runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_, featureWeights):
    with open( cfgTmpOutFileName, 'w+') as outFile:
        outFile.write("")
    args = []
    #print("indriRunQuery_hist =", indriRunQuery_hist)
    for (docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, cfgTmpOutFileName) in indriRunQuery_hist:
        #print("cfgTmpOutFileName =", cfgTmpOutFileName)
        if '_'.join([docno, ('_'.join(origWords_)).replace(' ', '_'), relW.replace(' ', '_')]) not in indriRunQuery_hist_:
            args += [(docno, origWords_, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)]
    #print("args =", args)
    if len(args)>0:
        genMultSimpQuery(args)
    cmd = ' '.join(["IndriRunQuery", cfgTmpOutFileName])
    #print("cmd =", cmd)
    indriRun = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #print("indriRun =", indriRun)
    indriRunQuery_hist_tmp = [[j for c, j in enumerate(i.split(' ')) if c in {0, 2, 4}] for i in indriRun.split('\n') if len(i)>0]
    #indriRunQuery_hist_ = dict()
    for i in indriRunQuery_hist_tmp:
        if i[0] in indriRunQuery_hist_:
            indriRunQuery_hist_[i[0]] += [[i[1], i[2]]]
        else:
            indriRunQuery_hist_[i[0]] = [[i[1], i[2]]]
    #indriRunQuery_hist_ = list(set(indriRunQuery_hist_))
    return indriRunQuery_hist_

In [130]:
def weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights):

    #print("'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') =", '_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_'))
    indriRunQuery_hist_1 = indriRunQuery_hist_['_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_')]
    #print("indriRunQuery_hist_1 =", indriRunQuery_hist_1)
    topTDocs = [i[0] for i in indriRunQuery_hist_1]
    #print("topTDocs =", topTDocs)
    
    expr = "#od4(" + regex.sub(' ', relW) + "):" + ','.join(topTDocs)
    cmd2 = ("efb", expr)
    #print("expr =", expr)
    if cmd2 not in dumpindex_hist:
        dumpindex_hist.add(cmd2)
    
    #print("topTDocs =", topTDocs)
    
    for documentName in topTDocs:
        cmd2 = ("dcf", docIdNameMap[documentName])
        if cmd2 not in dumpindex_hist:
            dumpindex_hist.add(cmd2)

    for ow in origWords_:
        cmd2 = ("efb", "#uw(#4( " + regex.sub(' ', ow) + " ) #4( " + regex.sub(' ', relW) + " )):" + ','.join(topTDocs))
        if cmd2 not in dumpindex_hist:
            dumpindex_hist.add(cmd2)
        
    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                cmd2 = ("efb", "#uw(#4( " + regex.sub(' ', ow1) + " ) #4( " + regex.sub(' ', ow2) + " ) #4( " + regex.sub(' ', relW) + " )):" + ','.join(topTDocs))
                if cmd2 not in dumpindex_hist:
                    dumpindex_hist.add(cmd2)
    return dumpindex_hist

In [131]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def runCommandsinHist_d(dist, docno, dumpindex_hist, dumpindex_hist_):
    args = []
    dumpindex_hist_dict = dict()
    tmp = set()
    for i in dumpindex_hist:
        if i[1] not in dumpindex_hist_.get(dist + "_" + docno + "_" + i[0], dict()):
            if i[1] in tmp:
                continue
            tmp.add(i[1])
            if i[0] in dumpindex_hist_dict:
                dumpindex_hist_dict[i[0]] += [[re.sub(ur"[^\w\d#(),:\-\s]+",' ',j) for j in i[1:]]]
            else:
                dumpindex_hist_dict[i[0]] =  [[re.sub(ur"[^\w\d#(),:\-\s]+",' ',j) for j in i[1:]]]
                
    for k, v in dumpindex_hist_dict.iteritems():
        fileName = dumpindexStatementFilename+"_"+k+".tmp"
        with open(fileName, 'w') as f:
            for i in v:
                f.write(' '.join(i) + '\n')
                #print("i =", i)
        cmd = ' '.join([occuranceCountFilename, colIndexDir, k, fileName])
        #print("cmd =", cmd)
        dumpindex_hist_tmp = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        #print("dumpindex_hist_tmp =", dumpindex_hist_tmp)
        dumpindex_hist_tmp = dict([tuple(j for c2, j in enumerate(i.split(':')) if c2 in {0,1} ) for i in dumpindex_hist_tmp.split("\n") if len(i)>0])
        #print("dumpindex_hist_tmp.get(\"#od4(travel)\",\"\") =", dumpindex_hist_tmp.get("#od4(travel)", dict()) )
        dumpindex_hist_tmp = {k1:v1.split(',') for k1,v1 in dumpindex_hist_tmp.iteritems()}
        dumpindex_hist_tmp = {k1:[i for i in v1 if len(i)>0] for k1,v1 in dumpindex_hist_tmp.iteritems()}
        #dumpindex_hist_tmp = {re.sub(ur"[^\w\d#()\s]+",' ',k1):[i for i in v1 if len(i)>0] for k1,v1 in dumpindex_hist_tmp.iteritems()}
        #dumpindex_tmp = [i for i in dumpindex_tmp.split("\n")]
        #print("dumpindex_hist_tmp =", dumpindex_hist_tmp)
        #dumpindex_hist_[k] = dumpindex_hist_tmp
        dumpindex_hist_[dist + "_" + docno + "_" + k] = dict(dumpindex_hist_.get(dist + "_" + docno + "_" + k, dict()).items() + dumpindex_hist_tmp.items())
    #print("dumpindex_hist_['efb'].get('#od4(travel)', dict())]", dumpindex_hist_['efb'].get("#od4(travel)", dict()))
        #print("dumpindex_hist_[k].keys() =", dumpindex_hist_[k].keys())
    #print("dumpindex_hist_ =", dumpindex_hist_)
        
    return dumpindex_hist_


In [149]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def documentLength(dist, docno, documentName):
    #cmd = ' '.join(["dumpindex", colIndexDir, "dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"])
    #cmd2 = ["dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"]
    #print("cmd =", cmd)
    #documentLength_ = int(dumpindex_colIndexDir(cmd2))
    documentLength_ = int(dumpindex_hist_[dist + "_" + docno + "_" + 'dcf'][documentName][0])
    return documentLength_

#print("docIdNameMap[\"FT944-15973\"] = ", docIdNameMap["FT944-15973"])
#print("documentLength(\"FT944-15973\") =", documentLength("FT944-15973"))

def weightRelConcept(dist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights):
    
    features = dict()
    
    ##########
    #cfgTmpOutFileName = os.path.join("/","tmp","sbsb",str(random.randint(1,1e6))+"_tmp.cfg")
    ##########
    #(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    #genOneSimpQuery(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    
    #indriRun = indriRunQuery_([cfgTmpOutFileName])
    #subprocess.Popen(' '.join(["rm", cfgTmpOutFileName]), shell=True, stdout=subprocess.PIPE).stdout.read()
    #indriRun = [i.split(' ') for i in indriRun.split('\n')]
    #indriRun = [i for i in indriRun if len(i)>1] # remove empty arrays
    print("'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') =", '_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_'))
    indriRunQuery_hist_1 = indriRunQuery_hist_['_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_')]
    #print("indriRunQuery_hist_1 =", indriRunQuery_hist_1)
    topTDocs = [i[0] for i in indriRunQuery_hist_1]
    
    #topTDocScores_d = {i[2]:np.float(i[4]) for i in indriRun}
    topTDocScores_d = {i[0]:np.float(i[1]) for i in indriRunQuery_hist_1}
    #print("topTDocScores_d =", topTDocScores_d)
    
    #scores = [ np.float(i[4]) for i in indriRun]
    #topTDocs = [i[2] for i in indriRun]

    #print("indriRun =", indriRun)
    #topTDocScore = np.sum([ np.float(i[4]) for i in indriRun])
    #expTDocScore = np.float(indriRun[0][4])
    expTDocScore = np.float(indriRunQuery_hist_1[0][1])
    features["expTDocScore"] = expTDocScore
    #print("expTDocScore =", features["expTDocScore"])
    
    #cmd =' '.join(["dumpindex", colIndexDir, "e", "\"#od4(" + relW + ")\""])
    #cmd2 = ["e", "\"#od4(" + relW + ")\""]
    #print("cmd =", cmd)
    #relWDocuments = dumpindex_colIndexDir(cmd2)
    expr = "#od4(" + regex.sub(' ', relW) + ")"
    #print ("dumpindex_hist_ =", dumpindex_hist_)
    #print ("expr =", expr)
    #print ("docno + '_' + 'efb' =", docno + '_' + 'efb')
    relWTopDocuments = list(dumpindex_hist_[dist + "_" + docno + "_" + 'efb'][expr])
    #print("relWTopDocuments =", relWTopDocuments)
    colTermFreq = np.float(relWTopDocuments[0])
    del relWTopDocuments[0] # the first element was colTermFreq
    #relWDocuments = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
    #relWDocuments = [i.split(' ') for i in relWDocuments.split('\n')]
    #print("relWDocuments =", relWDocuments)
    #relWDocuments = [[docNameIdMap[i[0]], i[1], i[2], i[3]] for j, i in enumerate(relWDocuments) if len(i)==4 and j>0] # remove empty arrays
    
    #relWTopDocuments = [i for i in relWDocuments if i[0] in topTDocs]
    #relWTopDocuments = [i for i in relWDocuments if i in topTDocs]
    #relWTopDocuments = [[i, topTDocScores_d[i]] for i in relWDocuments if i in topTDocs] # topdocs and their scores that contain relW
    #print("relWTopDocuments =", relWTopDocuments)
    #numerator = np.sum([int(i[1]) for i in relWTopDocuments]) # term count in the collection
    numerator = colTermFreq # relW term count in the collection
    #print("topTDocs =", topTDocs)
    denumerator = np.sum([documentLength(dist, docno, i) for i in topTDocs]) # number of terms in topDocs
    topTermFrac = numerator / np.float(denumerator)
    #print("numerator, denumerator, topTermFrac =", numerator, denumerator, topTermFrac)
    features["topTermFrac"] = topTermFrac
    
    numCanDocs = len(set([i[0] for i in relWTopDocuments]))
    features["numCanDocs"] = numCanDocs
    #print("numCanDocs =", features["numCanDocs"])
    
    if numCanDocs > 0:
        avgCDocScore = np.sum([ topTDocScores_d[i] for i in set([i for i in relWTopDocuments])])/np.float(numCanDocs)
    else:
        avgCDocScore = 0
    features["avgCDocScore"] = avgCDocScore
    #print("avgCDocScore =", features["avgCDocScore"])

    #for i in set([i[0] for i in relWTopDocuments]):
    #    print(i, topTDocScores_d[i])
    
    l = [ topTDocScores_d[i] for i in set(relWTopDocuments)]
    if len(l)>0:
        maxCDocScore = np.max(l)
    else:
        maxCDocScore = 0
    features["maxCDocScore"] = maxCDocScore
    #print("maxCDocScore =", features["maxCDocScore"])
    
    if colTermFreq>0:
        conIdf = np.log(colDocumentCount / colTermFreq)
    else:
        conIdf = np.log(colDocumentCount / 1)
    features["conIdf"] = conIdf
    #print("conIdf =", features["conIdf"])
    
    coocurDocuments = dict()
    coocurDocumentsTop = dict()
    for ow in origWords_:
        #cmd = ' '.join(["dumpindex", colIndexDir, "e", "\"#uw(#4( " + ow + " ) #4( " + relW + " ))\""])
        #cmd2 = ["e", "\"#uw(#4( " + ow + " ) #4( " + relW + " ))\""]
        #print("cmd =", cmd)
        #coocurDocuments_ = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
        #coocurDocuments_ = dumpindex_colIndexDir(cmd2)
        #coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
        coocurDocuments[ow] = [i for i in dumpindex_hist_[dist + "_" + docno + "_" + 'efb']["#uw(#4( " + regex.sub(' ', ow) + " ) #4( " + regex.sub(' ', relW) + " ))"]]
        #coocurDocuments[ow] = [docNameIdMap[i[0]] for i in coocurDocuments_ if len(i)==4] # remove empty arrays
        #print("topTDocs =", topTDocs)
        coocurDocumentsTop[ow] = [i for i in coocurDocuments[ow] if i in topTDocs ] # remove empty arrays

    #for k, v in coocurDocuments.iteritems():
    #    print(k, len(v))
        
    #for v in coocurDocumentsTop.values():
    #    print(v)
    #    print(len(set(v)))
        
    avgColCor = np.sum([len(set(i)) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColCor"] = avgColCor
    #print("avgColCor =", features["avgColCor"])
    
    maxColCor = np.max([len(set(i)) for i in coocurDocuments.values()])
    features["maxColCor"] = maxColCor
    #print("maxColCor =", features["maxColCor"])
    
    avgTopColCor = np.sum([len(set(i)) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColCor"] = avgTopColCor
    #print("avgTopColCor =", features["avgTopColCor"])
    
    l = [len(set(i)) for i in coocurDocumentsTop.values()]
    if len(l)>0:
        maxTopColCor = np.max(l)
    else:
        maxTopColCor = 0
    features["maxTopColCor"] = maxTopColCor
    #print("maxTopColCor =", features["maxTopColCor"])

    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                #coocurDocuments_ = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "e", "\"#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))\""]), shell=True, stdout=subprocess.PIPE).stdout.read()
                #cmd2 = [ "e", "\"#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))\""]
                #coocurDocuments_ = dumpindex_colIndexDir(cmd2)
                #coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
                coocurDocuments[ow1+ow2] = [i for i in dumpindex_hist_[dist + "_" + docno + "_" + 'efb']["#uw(#4( " + regex.sub(' ', ow1) + " ) #4( " + regex.sub(' ', ow2) + " ) #4( " + regex.sub(' ', relW) + " ))"]]
                #coocurDocuments[ow1+ow2] = [docNameIdMap[i[0]] for i in coocurDocuments_ if len(i)==4] # remove empty arrays
                coocurDocumentsTop[ow1+ow2] = [i for i in coocurDocuments[ow1+ow2] if i in topTDocs ] # remove empty arrays

    avgColPCor = np.sum([len(set(i)) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColPCor"] = avgColPCor
    #print("avgColPCor =", features["avgColPCor"])

    l = [len(set(i)) for i in coocurDocuments.values()]
    if len(l)>0:
        maxColPCor = np.max(l)
    else:
        maxColPCor = 0
    features["maxColPCor"] = maxColPCor
    #print("maxColPCor =", features["maxColPCor"])
    
    avgTopColPCor = np.sum([len(set(i)) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColPCor"] = avgTopColPCor
    #print("avgTopColPCor =", features["avgTopColPCor"])

    l = [len(set(i)) for i in coocurDocumentsTop.values()] 
    if len(l)>0:
        maxTopColPCor = np.max(l)
    else:
        maxTopColPCor = 0
    features["maxTopColPCor"] = maxTopColPCor
    #print("maxTopColPCor =", features["maxTopColPCor"])
    
    #print("docno, origWords_, relW =", docno, origWords_, relW)
    #print("features =", features)
    
    relWg = 0
    for feature, weight in featureWeights.iteritems():
        relWg += features[feature] * weight
    
    #print("features.keys() =", features.keys())
    return relWg


In [133]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords_train(args, indriRunQuery_hist_, dumpindex_hist_):
    #print("origWords =", origWords)
    text_string = ""
    
    count_history, rWords, dist, docno, origWords, expansionCount, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights = args
        
    relWords = dict()
    #if method == "relAll":
            
    if method in {"lr", "wsdmImpr"}:
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []
        for ow in origWords_:
            #print("ow =", ow)
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            indriRunQuery_hist = []
            for relW in conceptnet5RelAll[ow].keys():
                indriRunQuery_hist += weightRelConcept_hist_i(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
            #print("len(indriRunQuery_hist) =", len(indriRunQuery_hist))
            indriRunQuery_hist_ = runCommandsinHist_i(cfgTmpOutFileName, indriRunQuery_hist, indriRunQuery_hist_, featureWeights)
            #print("len(indriRunQuery_hist_) =", len(indriRunQuery_hist_))
            
            dumpindex_hist = set()
            for relW in conceptnet5RelAll[ow].keys():
                dumpindex_hist |= weightRelConcept_hist_d(indriRunQuery_hist_, dumpindex_hist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
            #print("dumpindex_hist =", dumpindex_hist)
            dumpindex_hist_ = runCommandsinHist_d(dist, docno, dumpindex_hist, dumpindex_hist_)
            #print("dumpindex_hist_ =", dumpindex_hist_)
            #print("len(dumpindex_hist_['ef']) =", len(dumpindex_hist_['ef']))
            #print("len(dumpindex_hist_['dcf']) =", len(dumpindex_hist_['dcf']))

    return(indriRunQuery_hist_, dumpindex_hist_)

In [141]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords(args, indriRunQuery_hist_, dumpindex_hist_):
    #print("origWords =", origWords)
    text_string = ""
    
    count_history, rWords, dist, docno, origWords, expansionCount, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights = args
        
    relWords = dict()
    #if method == "relAll":
        
    if method == "assocRestful":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        url = "http://conceptnet5.media.mit.edu/data/5.4/assoc/list/en/"+','.join(origWords_)
        response = urllib2.urlopen(url)
        data = response.read()
        data_j = json.loads(data)
        relWords = {str(d[0].encode('utf-8')).replace('/c/en/', '').replace('-', ' '):np.float(d[1]) for d in data_j[u'similar'] if '/c/en/' in str(d[0].encode('utf-8'))}
    
    elif method in {"lr", "wsdmImpr"}:
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []

        for ow in origWords_:
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            for relW in conceptnet5RelAll[ow].keys():
                relWg = weightRelConcept(dist, docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights)
                if relW in relWords:
                    relWords[relW] += relWg
                else:
                    relWords[relW] = relWg  
    
    else:
        count_history = countExpr(count_history, origWords, rWords)
        expressions = set()
        for ow in origWords:
            rWords_ow = rWords.get(ow, [])
            expressions.add("#4( " + ow.translate(None, string.punctuation) + " )")
            if ow.strip() == "":
                continue
            if simMeasure == "mi":
                N_w = countExpr_get_1(count_history, ow)
            for rw_wv in rWords_ow:
                (rw_w, rw_v) = rw_wv.items()[0]
                if rw_w.strip() == "":
                    continue           
                if simMeasure == "mi":
                    N_v = countExpr_get_1(count_history, rw_w)
                    N_wv = countExpr_get_2(count_history, ow, rw_w)
                    rw_v_ = mi_(N_w, N_v, N_wv)
                elif simMeasure == "cnet":
                    rw_v_ = rw_v
                relWords[rw_w] = relWords.get(rw_w, 0) + rw_v_
    
    relWords_sorted = sorted(relWords.items(), key=operator.itemgetter(1), reverse=True)
    #print ("origWords =", origWords)
    #print ("relWords_sorted =", relWords_sorted)
    #counter = 0
    relWords_sel = []
    for counter, (rw_w, rw_v) in enumerate(relWords_sorted):
        if (counter >= expansionCount):
            break
        if  all(c in string.printable for c in rw_w):
            if rw_w not in origWords:
                rw_w = regex.sub(' ', rw_w)
                text_string += rw_w + " "
                relWords_sel += [rw_w]
        #counter += 1
    #print("counter =", counter)
    #rint("relWords_sel =", relWords_sel)
#    return(relWords_sel)
    return(count_history, text_string, relWords_sel)

In [142]:
def getAllRelWords_train(dist, count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_):
    relText_stringAll = []
    relText_string = dict()
    relWords_sel = dict()
    for c1, (docno, origWords) in enumerate(origWordsAll.iteritems()):
        print("c1 =", c1, "docno =", docno, "--- origWords =", origWords)
        args = (count_history, rWords, dist, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        indriRunQuery_hist_, dumpindex_hist_ = getRelWords_train(args, indriRunQuery_hist_, dumpindex_hist_)
        #print("c1 =", c1, "- len(indriRunQuery_hist_), len(dumpindex_hist_.get('efb', dict())), len(dumpindex_hist_.get('dcf', dict())) =", len(indriRunQuery_hist_), len(dumpindex_hist_.get('efb', dict())), len(dumpindex_hist_.get('dcf', dict())))
    
    #print("relText_stringAll =", relText_stringAll)
    return (indriRunQuery_hist_, dumpindex_hist_)

In [143]:
def getAllRelWords(dist, count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_):
    relText_stringAll = []
    relText_string = dict()
    relWords_sel = dict()
    for c1, (docno, origWords) in enumerate(origWordsAll.iteritems()):
        #print(str(c1), "---", docno, "---", origWords)
        args = (count_history, rWords, dist, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        count_history, relText_string_, relWords_sel_ = getRelWords(args, indriRunQuery_hist_, dumpindex_hist_)
        #print("len(indriRunQuery_hist_), len(dumpindex_hist_['dcf']), len(dumpindex_hist_['ef']) =", len(indriRunQuery_hist_), len(dumpindex_hist_['dcf']), len(dumpindex_hist_['ef']))
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_[0:N1]
        #print(docno, "---", origWords, "---", relWords_sel_)
    relText_stringAll += [relText_string]
    """
    relText_string = dict()
    relWords_sel = dict()
    for docno, origWords_new in relWords_sel.iteritems():
        args = (count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
        count_history, relText_string_, relWords_sel_ = getRelWords(args, indriRunQuery_hist_, dumpindex_hist_)
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_
    relText_stringAll += [relText_string]
    """
    #print("relText_stringAll =", relText_stringAll)
    return (count_history, relText_stringAll, relWords_sel)

In [144]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genQueries(cfgOutFileName_, origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff):
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soupNew = BeautifulSoup("", 'lxml')
        parameters_tag = soupNew.new_tag("parameters")
        index_tag = soupNew.new_tag("index")
        index_tag.string = colIndexDir
        parameters_tag.append(index_tag)
            #print(tags)

        for docno, origWords in origWordsAll.iteritems():
            doc_tag = soupNew.new_tag("query")

            docno_tag = soupNew.new_tag("number")
            docno_tag.string = docno
            doc_tag.append(docno_tag)

            text_tag = soupNew.new_tag("text")
            text_tag.string = "#weight(\n" 
                    
            text_tag.string += str(intCoeff0) + " #combine(" 
            for ow in set(origWords):
                ow = regex.sub(' ', ow)
                text_tag.string += ow + " "
            text_tag.string += ")\n"

            if len(relText_stringAll[0])>0:
                if len(relText_stringAll[0][docno]) > 3: 
                    relText_string1 = regex.sub(' ', relText_stringAll[0][docno])
                    text_tag.string += str(intCoeff1) + " #combine(" 
                    #print("relText_string1 =", relText_string1)
                    text_tag.string += relText_string1.encode('utf-8')
                    text_tag.string += ")\n"
            
            """
            #print(len(relText_stringAll[1]))
            if len(relText_stringAll[1])>0:
                if len(relText_stringAll[1][docno]) > 3: 
                    relText_string2 = regex.sub(' ', relText_stringAll[1][docno])
                    text_tag.string += str(intCoeff2) + " #combine(" 
                    #print("relText_string2 =", relText_string2)
                    text_tag.string += relText_string2.encode('utf-8')
                    text_tag.string += ")\n"
            """
            #print("origWords =", origWords)
            #print("relWords_sel1 =", relWords_sel1)
            #print("relWords_sel2 =", relWords_sel2)
            
            
            text_tag.string += ") "

            doc_tag.append(text_tag)

            parameters_tag.append(doc_tag)
            #print(doc_tag)

        rule_tag = soupNew.new_tag("rule")
        rule_tag.string = "method:dir,mu:" + str(dirCoeff)
        #rule_tag.string = "method:two"
        parameters_tag.append(rule_tag)

        #intCoeff_tag = soupNew.new_tag("intCoeff")
        #intCoeff_tag.string = "0.8"
        #parameters_tag.append(intCoeff_tag)

        threads_tag = soupNew.new_tag("threads")
        threads_tag.string = "32"
        parameters_tag.append(threads_tag)

        count_tag = soupNew.new_tag("count")
        count_tag.string = "1000"
        parameters_tag.append(count_tag)

        trecFormat_tag = soupNew.new_tag("trecFormat")
        trecFormat_tag.string = "true"
        parameters_tag.append(trecFormat_tag)

        soupNew.append(parameters_tag)
        #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgOutFileName_, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)


In [145]:
def precisionCompute_train(dist, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_, dumpindex_hist_):
    #randNum = random.randint(1,1e9)
    origWordsAll = analyseQueries()
    indriRunQuery_hist_, dumpindex_hist_ = getAllRelWords_train(dist, count_history, rWords, origWordsAll, expansionCounts1[0], N1s[0], expansionCounts2[0], intCoeffs0[0], intCoeffs1[0], intCoeffs2[0], dirCoeffs[0], featureWeights_[0], indriRunQuery_hist_, dumpindex_hist_)
                                    
    return origWordsAll, count_history, indriRunQuery_hist_, dumpindex_hist_

In [146]:
#training
dist = str(1)
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.3 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 85 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [0]
rWords = dict()
featureWeights_1 = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
featureWeights_l=[featureWeights_1]
origWordsAll, count_history, indriRunQuery_hist_, dumpindex_hist_ = precisionCompute_train(dist, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_l, indriRunQuery_hist_, dumpindex_hist_)


c1 = 0 docno = 407 --- origWords = ['poach', 'wildlife', 'preserve']
c1 = 1 docno = 406 --- origWords = ["parkinson's", 'disease']
c1 = 2 docno = 405 --- origWords = ['cosmic', 'event']
c1 = 3 docno = 404 --- origWords = ['ireland', 'peace', 'talk']
c1 = 4 docno = 403 --- origWords = ['osteoporosis']
c1 = 5 docno = 402 --- origWords = ['behavioral', 'genetics']
c1 = 6 docno = 401 --- origWords = ['foreign', 'minority', 'germany']
c1 = 7 docno = 400 --- origWords = ['amazon', 'rain', 'forest']
c1 = 8 docno = 409 --- origWords = ['legal', 'pan', 'be', '103']
c1 = 9 docno = 408 --- origWords = ['tropical', 'storm']
c1 = 10 docno = 366 --- origWords = ['commercial', 'cyanide', 'use']
c1 = 11 docno = 420 --- origWords = ['carbon', 'monoxide', 'poison']
c1 = 12 docno = 364 --- origWords = ['rabies']
c1 = 13 docno = 422 --- origWords = ['art', 'steal', 'forge']
c1 = 14 docno = 425 --- origWords = ['counterfeit', 'money']
c1 = 15 docno = 363 --- origWords = ['transportation', 'tunnel', 'disast

In [151]:
def precisionCompute(dist, origWordsAll, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_, dumpindex_hist_):
    #mapPrecs = dict()
    for featureWeights in featureWeights_:
        for N1 in N1s:
            for intCoeff0 in intCoeffs0:
                for intCoeff1 in intCoeffs1:
                    for intCoeff2 in intCoeffs2:
                        for expansionCount1 in expansionCounts1:
                            for expansionCount2 in expansionCounts2:
                                for dirCoeff in dirCoeffs:
                                    
                                    
                                    #res = getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
                                    
                                    count_history, relText_stringAll, relWords_sel = getAllRelWords(dist, count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_)
                                    
                                    #print("len(relText_stringAll[1]) =", len(relText_stringAll[1]))
                                    ##########
                                    #cfgOutFileName = os.path.join("tmp","sbsb",str(randNum)+".cfg")
                                    #runsFileName = os.path.join("tmp","sbsb",str(randNum)+".run")
                                    #evalsFileName = os.path.join("tmp","sbsb",str(randNum)+".eval")
                                    ##########
                                    genQueries(cfgOutFileName, origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                                    subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                                    subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                                    #print ("cmd = ", cmd)
                                    mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    print("featureWeights =", featureWeights)
                                    print("intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, map precision, N1 =", intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1, mapPrec)
                                    #mapPrecs[intCoeff] = mapPrec
    return relText_stringAll, relWords_sel

In [ ]:
dist = str(1)
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.5 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 10 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s_max = [100]
#N1s = range(1, 30, 1)
#mapPrecs = 
rWords = dict()
featureWeights_l=[]
featureWeights_1 = {'maxTopColCor': 400, 'expTDocScore': 190, 'maxColPCor': 0.1, 'avgCDocScore': -40, 'topTermFrac': 0.1, 'avgColCor': 500, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
for conIdf in [50] :
#for avgCDocScore in np.arrange():
    featureWeights_1["conIdf"] = conIdf
    featureWeights_l += [dict(featureWeights_1)]
relText_stringAll, relWords_sel = precisionCompute(dist, origWordsAll, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s_max, featureWeights_l, indriRunQuery_hist_, dumpindex_hist_)


'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_illegally
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_boil_food
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_poach
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_muddy
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_hunt
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_illeagal
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_water
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_trespass
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_preserve_game
'_'.join([docno, '_'.join(origWords_), relW]).replace(' ', '_') = 407_poach_wildlife_pre

In [154]:
print(dumpindex_hist_.keys())

['380_dcf', '1_417_dcf', '360_dcf', '1_438_dcf', '1_364_efb', '404_dcf', '396_dcf', '365_efb', '1_356_dcf', '391_efb', '420_dcf', '444_dcf', '1_367_dcf', '426_efb', '361_dcf', '1_426_efb', '1_397_efb', '435_efb', '1_420_efb', '1_396_dcf', '1_356_efb', '441_dcf', '428_dcf', '1_353_dcf', '1_429_dcf', '1_416_dcf', '1_404_efb', '362_dcf', '447_efb', '1_360_efb', '414_efb', '352_dcf', '1_389_efb', '359_efb', '388_dcf', '413_efb', '419_efb', '1_417_efb', '1_372_dcf', '410_dcf', '357_dcf', '421_efb', '432_efb', '409_efb', '449_efb', '1_421_efb', '364_dcf', '372_dcf', '376_dcf', '386_dcf', '1_392_efb', '424_efb', '1_371_efb', '1_374_efb', '1_413_efb', '1_367_efb', '1_351_efb', '1_415_efb', '1_443_efb', '437_efb', '1_424_efb', '359_dcf', '1_365_efb', '1_413_dcf', '355_efb', '1_363_dcf', '1_376_efb', '1_403_efb', '1_439_efb', '1_357_dcf', '382_efb', '1_380_efb', '1_435_efb', '1_418_dcf', '1_433_dcf', '1_450_dcf', '405_efb', '1_448_efb', '1_388_efb', '1_398_efb', '1_358_efb', '1_353_efb', '398_ef

In [ ]:
def precisionCompute_train2(relWords_sel, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_, dumpindex_hist_):

    indriRunQuery_hist_, dumpindex_hist_ = getAllRelWords_train(count_history, rWords, relWords_sel, expansionCounts1[0], N1s[0], expansionCounts2[0], intCoeffs0[0], intCoeffs1[0], intCoeffs2[0], dirCoeffs[0], featureWeights_[0], indriRunQuery_hist_, dumpindex_hist_)
                                    
    return indriRunQuery_hist_, dumpindex_hist_

In [ ]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.5 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 10 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [2]
rWords = dict()
featureWeights_1 = {'maxTopColCor': 0.1, 'expTDocScore': 0.1, 'maxColPCor': 0.1, 'avgCDocScore': 0.1, 'topTermFrac': 0.1, 'avgColCor': 0.1, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
featureWeights_l=[featureWeights_1]
indriRunQuery_hist_, dumpindex_hist_ = precisionCompute_train2(relWords_sel, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_l, indriRunQuery_hist_, dumpindex_hist_)


In [ ]:
def precisionCompute2(relWords_sel, origWordsAll, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_, indriRunQuery_hist_, dumpindex_hist_):
    #mapPrecs = dict()
    for featureWeights in featureWeights_:
        for N1 in N1s:
            for intCoeff0 in intCoeffs0:
                for intCoeff1 in intCoeffs1:
                    for intCoeff2 in intCoeffs2:
                        for expansionCount1 in expansionCounts1:
                            for expansionCount2 in expansionCounts2:
                                for dirCoeff in dirCoeffs:
                                    
                                    
                                    #res = getAllRelWords_p(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights)
                                    
                                    count_history, relText_stringAll2, relWords_sel2 = getAllRelWords(count_history, rWords, relWords_sel, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff, featureWeights, indriRunQuery_hist_, dumpindex_hist_)
                                    
                                    print("relWords_sel2 =", relWords_sel2)
                                    
                                    #print("len(relText_stringAll[1]) =", len(relText_stringAll[1]))
                                    ##########
                                    #cfgOutFileName = os.path.join("tmp","sbsb",str(randNum)+".cfg")
                                    #runsFileName = os.path.join("tmp","sbsb",str(randNum)+".run")
                                    #evalsFileName = os.path.join("tmp","sbsb",str(randNum)+".eval")
                                    ##########
                                    genQueries(cfgOutFileName, origWordsAll, relWords_sel2, relText_stringAll, relText_stringAll2, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                                    subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                                    subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                                    #print ("cmd = ", cmd)
                                    mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                    print("featureWeights =", featureWeights)
                                    print("intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, map precision, N1 =", intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1, mapPrec)
                                    #mapPrecs[intCoeff] = mapPrec
    return relWords_sel2

In [ ]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.5 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 10 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [3]
#N1s = range(1, 30, 1)
#mapPrecs = 
rWords = dict()
featureWeights_l=[]
featureWeights_1 = {'maxTopColCor': 400, 'expTDocScore': 190, 'maxColPCor': 0.1, 'avgCDocScore': -40, 'topTermFrac': 0.1, 'avgColCor': 500, 'avgTopColCor': 0.1, 'numCanDocs': 0.1, 'maxTopColPCor': 0.1, 'maxCDocScore': 0.1, 'avgColPCor': 0.1, 'conIdf': 0.1, 'avgTopColPCor': 0.1, 'maxColCor': 0.1}
for conIdf in [50] :
#for avgCDocScore in np.arrange():
    featureWeights_1["conIdf"] = conIdf
    featureWeights_l += [dict(featureWeights_1)]
relWords_sel2 = precisionCompute2(relWords_sel, origWordsAll, count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s, featureWeights_l, indriRunQuery_hist_, dumpindex_hist_)


In [ ]:
print(indriRunQuery_hist_)


In [ ]:
print(relWords_sel)